# 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
import warnings
warnings.filterwarnings("ignore")

# 데이터 불러오기

In [2]:
data_path: str = "~/house/data"
train_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv"))
test_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))
sample_submission: pd.DataFrame = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [3]:
# 금리, 지하철, 학교, 공원 정보 불러오기
interest_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "interestRate.csv"))
subway_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "subwayInfo.csv"))
school_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "schoolinfo.csv"))
park_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "parkInfo.csv"))

# 데이터 병합

## 금리 데이터 병합
* `interest_data`: 2018년 12월 ~ 2024년 5월까지의 금리
* 계약 연월 기준으로 `interest_data`를 `train_data`로 병합 (2019년 4월 ~ 2023년 12월)
* 계약 연월 기준으로 `interest_data`를 `test_data`로 병합 (2024년 1월 ~ 2024년 6월)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
interest_data

In [ ]:
# 계약 연월 기준으로 interest_data를 train_data로 병합
merged_train = pd.merge(train_data, interest_data, left_on="contract_year_month", right_on="year_month", how="left")
merged_train = merged_train.drop(columns=["year_month"])
merged_train

In [ ]:
merged_test = pd.merge(test_data, interest_data, left_on="contract_year_month", right_on="year_month", how="left")
merged_test = merged_test.drop(columns=["year_month"])
merged_test

In [ ]:
# 금리 결측치 개수 확인 (2024년 6월)
merged_test["interest_rate"].isnull().sum()

## 최단거리 데이터 병합

### sklearn의 BallTree를 활용한 haversine 거리 계산 함수

In [10]:
def find_closest_distance_haversine(
    train_data: pd.DataFrame, 
    loc_df: pd.DataFrame
) -> pd.DataFrame:
    """건물과 지하철/학교/공원 사이의 haversine 거리를 계산하는 함수

    Args:
        train_data (pd.DataFrame): 학습(훈련) 또는 테스트 데이터프레임
        loc_df (pd.DataFrame): 위도, 경도를 column으로 갖는 데이터프레임

    Returns:
        pd.DataFrame: index, 위도, 경도, haversine 거리를 column으로 갖는 반환
    """
    # degree->radian 값으로 변환 for 삼각함수
    train_coords = np.radians(train_data[["latitude", "longitude"]].values)
    loc_coords = np.radians(loc_df[["latitude", "longitude"]].values)
    
    # Ball Tree 생성 
    tree = BallTree(loc_coords, metric="haversine")

    distances, indices = tree.query(train_coords, k=1) # 가까운 1 지점만 
    distances_meter = distances * 6371000 # 단위를 meter로 변환

    closest_coords = loc_df[["latitude", "longitude"]].iloc[indices.flatten()].values # 가까운 지점 좌표

    # index, 최단거리, 최단거리에 해당하는 지점의 위도, 경도로 이루어진 데이터프레임 생성
    result_df = pd.DataFrame({
        "index" : train_data.index,
        "closest_distance" : distances_meter.flatten(),
        "closest_latitude" : closest_coords[:, 0],
        "closest_longtitude" : closest_coords[:, 1]
    })

    return result_df


### subway 병합

In [11]:
subway_result = find_closest_distance_haversine(train_data, subway_data)
subway_result.columns = ["index", "nearest_subway_distance", "nearest_subway_latitude", "nearest_subway_longtitude"]
train_data = pd.merge(train_data, subway_result, on="index")

In [12]:
subway_result = find_closest_distance_haversine(test_data, subway_data)
subway_result.columns = ["index", "nearest_subway_distance", "nearest_subway_latitude", "nearest_subway_longtitude"]
test_data = pd.merge(test_data, subway_result, on="index")

### school 병합

In [13]:
school_result = find_closest_distance_haversine(train_data, school_data)
school_result.columns = ["index", "nearest_school_distance", "nearest_school_latitude", "nearest_school_longtitude"]
train_data = pd.merge(train_data, school_result, on="index")

In [14]:
school_result = find_closest_distance_haversine(test_data, school_data)
school_result.columns = ["index", "nearest_school_distance", "nearest_school_latitude", "nearest_school_longtitude"]
test_data = pd.merge(test_data, school_result, on="index")

### park 병합

In [15]:
park_result = find_closest_distance_haversine(train_data, park_data)
park_result.columns = ["index", "nearest_park_distance", "nearest_park_latitude", "nearest_park_longtitude"]
train_data = pd.merge(train_data, park_result, on="index")

In [16]:
park_result = find_closest_distance_haversine(test_data, park_data)
park_result.columns = ["index", "nearest_park_distance", "nearest_park_latitude", "nearest_park_longtitude"]
test_data = pd.merge(test_data, park_result, on="index")

## 병합한 데이터 확인

In [ ]:
on = merged_train.columns.drop("interest_rate").tolist() # 병합 기준이 될 column 리스트
train_data = pd.merge(merged_train, train_data, on=on, how="left")
# train_data = train_data.drop(columns=["index"])
train_data

In [ ]:
on = merged_test.columns.drop("interest_rate").tolist() # 병합 기준이 될 column 리스트
test_data = pd.merge(merged_test, test_data, on=on, how="left")
# test_data = test_data.drop(columns=["index"])
test_data

# Data Preprocessing

In [ ]:
# built_year > 2024 행 삭제
print("before train :", train_data.shape)
train_data = train_data[train_data["built_year"] < 2024]
train_data.reset_index(drop=True, inplace=True)
print("after train :", train_data.shape)

In [ ]:
train_data["built_year"]

### 클러스터링

In [21]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

### 엘보우 방법(Elbow Method)
- 클러스터 수에 따른 WCSS(Within-Cluster Sum of Squares)를 계산하고, 그래프에서 급격한 변화가 있는 지점을 찾음
- 이 지점이 적절한 클러스터 수
- 여기서는 3

In [ ]:
coords = train_data[['latitude', 'longitude']]
wcss = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(coords)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 20), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

### Elbow Method로 찾은 n_cluster 적용

In [ ]:
# KMeans 학습 (train 데이터에서)
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(train_data[['latitude', 'longitude']])

# 학습된 모델 저장
joblib.dump(kmeans, 'kmeans_model.pkl')

# 학습된 모델로 train 데이터에 레이블 할당
train_data['region'] = kmeans.predict(train_data[['latitude', 'longitude']])

# 지역별 전세가 분포 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='region', y='deposit', data=train_data)
plt.title('Deposit Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Deposit')
plt.show()

In [25]:
# 저장된 KMeans 모델 불러와서 test 데이터에 레이블 할당
kmeans = joblib.load('kmeans_model.pkl')

# Test 데이터에 동일한 KMeans 모델 적용
test_data['region'] = kmeans.predict(test_data[['latitude', 'longitude']])


In [ ]:
# 클러스터별 색상 지정
colors = {0: 'red', 1: 'blue', 2: 'green'}  # 3개의 클러스터에 각각 색상 지정

# 시각화
plt.figure(figsize=(10, 6))

# 각 클러스터에 해당하는 점들을 색상별로 플롯
for cluster, color in colors.items():
    clustered_data = train_data[train_data['region'] == cluster]
    plt.scatter(clustered_data['longitude'], clustered_data['latitude'], 
                c=color, label=f'Region {cluster}', alpha=0.5, s=10)

# 제목 및 축 레이블 설정
plt.title('KMeans Clustering of Regions (Based on Latitude and Longitude)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# 범례 추가
plt.legend()

# 시각화 표시
plt.show()

In [ ]:
# 클러스터별 색상 지정
colors = {0: 'red', 1: 'blue', 2: 'green'}  # 3개의 클러스터에 각각 색상 지정

# 시각화
plt.figure(figsize=(10, 6))

# 각 클러스터에 해당하는 점들을 색상별로 플롯
for cluster, color in colors.items():
    clustered_data = test_data[test_data['region'] == cluster]
    plt.scatter(clustered_data['longitude'], clustered_data['latitude'], 
                c=color, label=f'Region {cluster}', alpha=0.5, s=10)

# 제목 및 축 레이블 설정
plt.title('KMeans Clustering of Regions (Based on Latitude and Longitude)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# 범례 추가
plt.legend()

# 시각화 표시
plt.show()

# Feature Engineering

In [28]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import optuna
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt

### 1. log 변환

In [29]:
train_data["log_deposit"] = np.log1p(train_data["deposit"])
train_data["log_area_m2"] = np.log1p(train_data["area_m2"])
train_data["log_school_distance"] = np.log1p(train_data["nearest_school_distance"])
train_data["log_park_distance"] = np.log1p(train_data["nearest_park_distance"])
train_data["log_subway_distance"] = np.log1p(train_data["nearest_subway_distance"])

In [30]:
test_data["log_area_m2"] = np.log1p(test_data["area_m2"])
test_data["log_school_distance"] = np.log1p(test_data["nearest_school_distance"])
test_data["log_park_distance"] = np.log1p(test_data["nearest_park_distance"])
test_data["log_subway_distance"] = np.log1p(test_data["nearest_subway_distance"])

### 2. Feature Select

In [31]:
# 피처 및 타겟 설정
train_cols = [
    "deposit",
    "log_deposit",
    "log_area_m2",
    "built_year",
    "latitude",
    "longitude",
    "log_subway_distance",
    "log_school_distance",
    "log_park_distance",
    "contract_year_month",
    "contract_day",
    "region"
]
train_data = train_data[train_cols]

In [32]:
# 피처 및 타겟 설정
test_cols = [
    "log_area_m2",
    "built_year",
    "latitude",
    "longitude",
    "log_subway_distance",
    "log_school_distance",
    "log_park_distance",
    "contract_year_month",
    "contract_day",
    "region"
]
test_data = test_data[test_cols]

In [ ]:
test_data

### 3. 데이터 분리

In [34]:
X = train_data.drop(columns=["deposit", "log_deposit"], inplace=False)
y = train_data[["deposit", "log_deposit"]]

# Modeling

## Optuna

In [35]:
best_params = {"n_estimators": 288, "learning_rate": 0.11112043349923437, "max_depth": 10, "subsample": 0.7511206505586165}
#이전 파라미터 {'n_estimators': 289, 'learning_rate': 0.18917689569941643, 'max_depth': 10, 'subsample': 0.815154297680078}

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

mae = []
for train_idx, valid_idx in kfold.split(X, train_data["deposit"]):
    X_train, y_train = X.loc[train_idx, :], y.loc[train_idx, "log_deposit"]
    X_valid, y_valid = X.loc[valid_idx, :], y.loc[valid_idx, "deposit"]
    best_model = xgb.XGBRegressor(**best_params, tree_method="gpu_hist", gpu_id=0, random_state=42)
    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_valid)
    y_pred = np.expm1(y_pred)
    mae.append(mean_absolute_error(y_valid, y_pred))

print(f"{np.mean(mae):.4f}")

In [ ]:
train_data

In [ ]:
test_data

# Inference

In [39]:
y_test_log = best_model.predict(test_data)
y_test = np.expm1(y_test_log) # 지수변환 (로그변환의 역변환)

In [ ]:
# 지수변환 전/후 예측값 히스토그램 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.histplot(y_test_log, ax=axes[0])
sns.histplot(y_test, ax=axes[1])
plt.show()

In [41]:
sample_submission["deposit"] = y_test
sample_submission.to_csv("output.csv", index=False)